In [1]:
import pandas, re, os, datetime

In [2]:
path_TOPOVN = "../../Data_SNP_Intern/TOPO_DATA_DESCRIPTION.xlsx"
file = pandas.ExcelFile(path_TOPOVN)
sheets = file.sheet_names
sheets.remove('TOPOVN')

In [131]:
import pyodbc
str_connection = 'Driver={SQL Server};Server=%s;Database=%s;Trusted_Connection=yes;'%('localhost', 'ThucTapSNP')
connection = pyodbc.connect(str_connection)
cursor = connection.cursor()

In [132]:
def CleanText(text):
    text = text.replace('\n', '\r\n')
    text = text.replace('_x000D_', '')
    text = text.replace('"', '')
    return text
def GetColumnName(lst):
    patt_col = 'column(.)*'
    patt_data = 'data type'
    col_columnName = list(filter(lambda x: re.match(patt_col, x.lower()), lst))
    col_datatype = list(filter(lambda x: re.match(patt_data, x.lower()), lst))
    return [col_columnName.pop(0), col_datatype.pop(0)]
def ConvertToSQLDatatype(text):
    text = text.lower().replace(' ', '')
    dct = {
      "number\([0-9]+,[0-9]+\)": ["number", "numeric"],
      "date": ["date", "datetime"],
      "byte.$": ["byte", ""],
      "^number\([0-9]\)$": ["", "int"],
      "^varchar2": ["varchar2", "varchar"]
    }
    for key in dct.keys():
        matched = re.findall(key, text)
        if len(matched) > 0:
            replaced, replace_with = dct[key]
            if replaced == '':
                text = replace_with
            else:
                text = text.replace(replaced, replace_with)
    return text

# def ConvertData(df, cols, lst):
#     for i in range(len(lst)):
#         try:
#             if lst[i] == 'datetime':
#                 for row in range(len(df)):
#                     date = df.loc[row, cols[i]]
#                     try:
#                         date = pandas.to_datetime(date)
#                     except pandas.errors.OutOfBoundsDatetime as error:
#                         date = datetime.datetime(1900, 12, 31, 23)
#                     except pandas.errors.ParserError as parser_error:
#                         df = df.drop(df.index[row])
#                         df = df.reset_index()
#                     df.loc[row, cols[i]] = date
#                 df = df.astype({cols[i]:"datetime64"})
# #             elif lst[i] == "int":
# #                 df[cols[i]] = df[cols[i]].apply(int)
#             elif 'numeric' in lst[i]:
#                 df[cols[i]] = df[cols[i]].apply(float)
#             elif 'char' in lst[i]:
#                 df[cols[i]] = df[cols[i]].apply(str)
# #                 df[cols[i]] = df[cols[i]].apply(lambda x: "'%s'"%(x))
#         except ValueError as value_error:
#             raise ValueError(' - '.join([cols[i], lst[i],"datetime"]))
#         except TypeError as type_error:
#             raise ValueError(' - '.join([cols[i], lst[i],"datetime"]))
#     return df
def ConvertData(data, col_columns, col_datatype):
    for i in range(len(col_datatype)):
        if col_datatype[i] == 'datetime':
            row = 0
            while row < len(data):
                try:
                    date = data.loc[row, col_columns[i]]
                    date = pandas.to_datetime(date)
                    data.loc[row, col_columns[i]] = date
                except pandas.errors.OutOfBoundsDatetime as error_outofdatetime:
                    data.loc[row, col_columns[i]] = datetime.datetime(1900, 12, 31, 23)
                # except pandas.errors.ParserError as error_parses:
                except:
                    data = data.drop(data.index[row])
                    data = data.reset_index()
                row += 1
            data = data.astype({col_columns[i] : 'datetime64'})
        elif 'char' in col_datatype[i]:
            data[col_columns[i]] = data[col_columns[i]].apply(CleanText)
        elif 'numeric' in col_datatype[i]:
            data[col_columns[i]] = data[col_columns[i]].apply(float)
    return data

In [134]:
for name in sheets:
    data = pandas.read_excel(file, sheet_name=name, header=1)
    columns = GetColumnName(list(data.columns))

    data = data.dropna(subset=[columns[0]])
    col_columns = [name.replace(' ', '_') for name in data[columns[0]].tolist()]
    col_datatype = [ConvertToSQLDatatype(i) for i in data[columns[1]].tolist()]
    
    data = pandas.read_excel(path_data + dct_fileAndData[name])
    i = 0
    while i < len(col_columns):
        if col_columns[i] not in list(data.columns):
            col_columns.remove(col_columns[i])
            col_datatype.remove(col_datatype[i])
        else:
            i += 1
        
    data = data.fillna('null')
    data, col_columns, col_datatype = CheckDatatype(data, col_columns, col_datatype)

    data = ConvertData(data, col_columns, col_datatype)
    data = data[col_columns]
    
    sql_tablebody = ''
    for i in range(len(col_columns)):
        sql_tablebody += col_columns[i] + ' ' + col_datatype[i] + ',\n'
    command = 'create table {name} ({body})\n'.format(name=name, body=sql_tablebody)
    cursor.execute(command)
    connection.commit()
    print(name, " created")
    
    command = 'insert into {table}({columns}) values ({values})'
    str_columns = ','.join(col_columns)
    for i in range(len(data)):
        row = ConvertListToSQLInsert(data.loc[i].tolist(), col_datatype)
#         print(command.format(
#             table = name,
#             columns = str_columns,
#             values = ','.join(row)
#         ))
        cursor.execute(command.format(
            table = name,
            columns = str_columns,
            values = ','.join(row)
        ))
#         break
    connection.commit()
#     break
    print(name, " inserted")

VESSEL_DETAILS  created
VESSEL_DETAILS  inserted
ITEM  created
ITEM  inserted
ITEM_LOCATION  created
ITEM_LOCATION  inserted
ITEM_REEFER  created
ITEM_REEFER  inserted
ITEM_OOG  created
ITEM_OOG  inserted
ITEM_DANGEROUS  created
ITEM_DANGEROUS  inserted
PREGATE_TRANSACT  created
PREGATE_TRANSACT  inserted
DISCHARGE_LIST  created
DISCHARGE_LIST  inserted
EDI_ITEM_PRE_ADVICE  created
EDI_ITEM_PRE_ADVICE  inserted
OPERATION_METHOD  created
OPERATION_METHOD  inserted
YARD_AREA  created
YARD_AREA  inserted


In [32]:
path_data = "../../Data_SNP_Intern/"
lstData = os.listdir(path_data)
lstData.remove("TOPO_DATA_DESCRIPTION.xlsx")

In [90]:
data['COMMODITY'].replace('"', '')

0        "SHIPPERS LOAD, COUNT & SEAL"\n"SAID TO CONTAI...
1        TOTAL:563 CARTONS_x000D_\nELECTRIC GRILL BN03#...
2        1X20 & 2X40 CONTAINER STC: 80 PALLETS (2400 BA...
3        TOTAL:660 PALLETS_x000D_\nCOMMODITY: PHOTOVOLT...
4                                    STAINLESS STEEL COILS
                               ...                        
30744                            KNIT FABRIC & ACCESSORIES
30745    TOTAL:838 ROLLS_x000D_\nUPHOLSTERY FABRIC_x000...
30746    TOTAL:162 ROLLS_x000D_\nEVA_x000D_\nHS CODE:39...
30747                                 FABRICS NOS TEXTILES
30748                        DAIRY PRODUCTS NON-FROZEN NOS
Name: COMMODITY, Length: 30749, dtype: object

In [33]:
dct_fileAndData = dict.fromkeys(sheets)

In [34]:
for key in dct_fileAndData.keys():
    for i in range(len(lstData)):
        if (key + ".xlsx") in lstData[i]:
            dct_fileAndData[key] = lstData[i]

In [35]:
dct_fileAndData

{'VESSEL_DETAILS': '1_VESSEL_DETAILS.xlsx',
 'ITEM': '2_ITEM.xlsx',
 'ITEM_LOCATION': '3_ITEM_LOCATION.xlsx',
 'ITEM_REEFER': '4_ITEM_REEFER.xlsx',
 'ITEM_OOG': '5_ITEM_OOG.xlsx',
 'ITEM_DANGEROUS': '6_ITEM_DANGEROUS.xlsx',
 'PREGATE_TRANSACT': '7_PREGATE_TRANSACT.xlsx',
 'DISCHARGE_LIST': '8_DISCHARGE_LIST.xlsx',
 'EDI_ITEM_PRE_ADVICE': '9_EDI_ITEM_PRE_ADVICE.xlsx',
 'OPERATION_METHOD': '10_OPERATION_METHOD.xlsx',
 'YARD_AREA': '11_YARD_AREA.xlsx'}

In [18]:
data = pandas.read_excel("../../Data_SNP_Intern/1_VESSEL_DETAILS.xlsx")

In [64]:
command = 'insert into {table}({columns}) values ({values})'

In [342]:
sheets

['VESSEL_DETAILS',
 'ITEM',
 'ITEM_LOCATION',
 'ITEM_REEFER',
 'ITEM_OOG',
 'ITEM_DANGEROUS',
 'PREGATE_TRANSACT',
 'DISCHARGE_LIST',
 'EDI_ITEM_PRE_ADVICE',
 'OPERATION_METHOD',
 'YARD_AREA']

In [68]:
import datetime
datetime.datetime.strptime("9220/10/11 20:12:00", "%y/%m/%d %H:%M:%S")

ValueError: time data '9220/10/11 20:12:00' does not match format '%y/%m/%d %H:%M:%S'

In [70]:
from dateutil import parser
date = parser.parse("9220/10/11 20:12:00")

In [5]:
command = "insert into {table}({columns}) values ({values})"

In [6]:
command.format(table="a", columns="dsad", values="dsa")

'insert into a(dsad) values (dsa)'

In [29]:
[str(i) + str(row.tolist()[i]) + "    " + col_datatype[i] for i in range(len(col_datatype))]

['0GSL-00022    char(12)',
 '1CTL    char(6)',
 '2GSLK2806N    char(12)',
 '3GSLK2806N    char(12)',
 '4GSL    char(4)',
 '5GLS PHUONG NAM    char(30)',
 '61900-12-31 23:00:00    datetime',
 '7SALAN    char(10)',
 '8Z    char(1)',
 '9B    char(1)',
 '10SG7984    char(20)',
 '11K2806N    char(12)',
 '12K2806N    char(12)',
 '13TCC    char(7)',
 '14nan    char(3)',
 '15CTL    char(6)',
 '16B08    char(5)',
 '171900-12-31 23:00:00    datetime',
 '182019-06-29 07:59:00    datetime',
 '192019-06-29 08:43:00    datetime',
 '202019-06-29 08:42:00    datetime',
 '211900-12-31 00:00:00    datetime',
 '222019-06-29 08:00:00    datetime',
 '23D    char(1)',
 '24nan    char(1)',
 '250    numeric(9,3)',
 '261900-12-31 23:00:00    datetime',
 '270    int',
 '28nan    char(5)',
 '290    int',
 '30nan    char(1)',
 '31P    char(1)',
 '32nan    char(5)',
 '33nan    char(6)',
 '341900-12-31 23:00:00    datetime',
 '350    numeric(9,3)',
 '361900-12-31 23:00:00    datetime',
 '372019-06-28 16:00:00    da

In [94]:
data = pandas.read_excel(file, sheet_name=sheets[7], header=1)
columns = GetColumnName(list(data.columns))

In [95]:
data = data.dropna(subset=[columns[0]])
col_columns = [name.replace(' ', '_') for name in data[columns[0]].tolist()]
col_datatype = [ConvertToSQLDatatype(i) for i in data[columns[1]].tolist()]

In [96]:
data = pandas.read_excel(path_data + dct_fileAndData[sheets[7]])
data = data.fillna('null')
data = data[col_columns]

In [97]:
data, col_columns, col_datatype = CheckDatatype(data, col_columns, col_datatype)

In [98]:
data = ConvertData(data, col_columns, col_datatype)

In [59]:
command = 'insert into {table}({columns}) values ({values})'.format(
        table=sheets[0],
        columns = ','.join(col_columns),
    values = ','.join(ConvertListToSQLInsert(row, col_datatype))
    )

In [54]:
checked = False
while not checked:
    print("Case")
    try:
        data = data[col_columns]
        checked = True
    except KeyError as error:
        text = str(error)
        col = str(text[text.index('[')+2 : text.index(']')-1])
        col_columns.remove(col)
    except: pass

Case
SITE_ID
['ITEM_KEY', 'DGS_CLASS', 'UN_NO', 'CHARGE_FLG']
Case


In [85]:
result = []
for char in data.loc[0, 'COMMODITY']:
    if char.isalpha():
        result.append(char)

In [87]:
data.loc[0, 'COMMODITY']

'"SHIPPERS LOAD, COUNT & SEAL"\n"SAID TO CONTAIN"\n1X20\n1,500 CARTONS\n\n15,0'

In [105]:
[col_columns[i] + " : " + col_datatype[i] for i in range(len(col_columns))]

['ITEM_KEY : int',
 'ITEM_NO : char(12)',
 'SITE_ID : char(6)',
 'ARR_BY : char(1)',
 'ARR_CAR : char(12)',
 'ARR_TS : datetime',
 'FEL : char(1)',
 'DEP_TS : datetime',
 'DEP_BY : char(1)',
 'DEP_CAR : char(12)',
 'ISO : char(1)',
 'LENGTH : numeric(9,3)',
 'CATEGORY : char(1)',
 'OWNER : char(3)',
 'EXIT_VES_CD : char(3)',
 'EXIT_VOYAGE : char(12)',
 'PLACE_OF_DELIVERY : char(8)',
 'PLACE_OF_RECEIPT : char(8)',
 'ITEM_TYPE : char(2)',
 'SLOT_CODE : char(7)',
 'LINE_OPER : char(7)',
 'AGENT : char(3)',
 'DISCH_PORT : char(5)',
 'FDISCH_PORT : char(5)',
 'HIST_FLG : char(1)',
 'LOAD_PORT : char(5)',
 'LL_DISCH_PORT : char(5)']

In [53]:
col

"'SITE_ID'"

In [34]:
for i in range(len(col_datatype)):
    if col_datatype[i] == 'datetime':
        row[i] = str(row[i])

In [105]:
col_columns[18]

'ACT_BERTH_TS'

In [106]:
col_datatype[18]

'datetime'

In [48]:
'SITE_ID' in col_columns

True

In [25]:
data = data.astype({'ACT_BERTH_TS':'datetime64'})

In [41]:
data.loc[0, 'ACT_BERTH_TS'].to_pydatetime().ipynb_checkpoints/

TypeError: int() argument must be a string, a bytes-like object or a number, not 'datetime.datetime'

In [98]:
data['ISO'].apply(lambda x: str(x))

0        2200
1        2200
2        4200
3        4200
4        4500
         ... 
47995    2200
47996    2200
47997    4500
47998    2200
47999    4500
Name: ISO, Length: 48000, dtype: object

In [32]:
data = data.fillna("null")

In [18]:
df = pandas.read_excel(path_data + dct_fileAndData[sheets[0]])

In [269]:
data = data.drop(data.index(0))

TypeError: 'RangeIndex' object is not callable

In [128]:
def ConvertListToSQLInsert(row, datatype):
    result = []
    for i in range(len(datatype)):
        if row[i] == 'null':
            result.append(row[i])
        elif 'varchar' in datatype[i]:
            result.append("N'%s'"%(row[i]))
        elif 'char' in datatype[i]:
            result.append("'%s'"%(row[i]))
        elif datatype[i] == 'datetime':
            result.append("'%s'"%(str(row[i])))
        else:
            result.append(str(row[i]))
    return result

In [50]:
','.join(ConvertListToSQLInsert(row, col_datatype))

"'GSL-00022','CTL','GSLK2806N','GSLK2806N','GSL','GLS PHUONG NAM','1900-12-31 23:00:00','SALAN','Z','B','SG7984','K2806N','K2806N','TCC',null,'CTL','B08','1900-12-31 23:00:00','2019-06-29 07:59:00','2019-06-29 08:43:00','2019-06-29 08:42:00','1900-12-31 00:00:00','2019-06-29 08:00:00','D',null,0.0,'1900-12-31 23:00:00',0,null,0,null,'P',null,null,'1900-12-31 23:00:00',0.0,'1900-12-31 23:00:00','2019-06-28 16:00:00','2019-06-28 19:00:00'"

In [48]:
ConvertListToSQLInsert(row, col_datatype)

["'GSL-00022'",
 "'CTL'",
 "'GSLK2806N'",
 "'GSLK2806N'",
 "'GSL'",
 "'GLS PHUONG NAM'",
 "'1900-12-31 23:00:00'",
 "'SALAN'",
 "'Z'",
 "'B'",
 "'SG7984'",
 "'K2806N'",
 "'K2806N'",
 "'TCC'",
 'null',
 "'CTL'",
 "'B08'",
 "'1900-12-31 23:00:00'",
 "'2019-06-29 07:59:00'",
 "'2019-06-29 08:43:00'",
 "'2019-06-29 08:42:00'",
 "'1900-12-31 00:00:00'",
 "'2019-06-29 08:00:00'",
 "'D'",
 'null',
 0.0,
 "'1900-12-31 23:00:00'",
 0,
 'null',
 0,
 'null',
 "'P'",
 'null',
 'null',
 "'1900-12-31 23:00:00'",
 0.0,
 "'1900-12-31 23:00:00'",
 "'2019-06-28 16:00:00'",
 "'2019-06-28 19:00:00'"]

In [37]:
checked = False
while not checked:
    try:
        data = data[col_columns]
        checked = True
    except KeyError as error:
        text = str(error)
        col = text[text.index('[')+1 : text.index(']')]
        col_columns.remove(col)

ValueError: list.remove(x): x not in list

In [74]:
for i in range(len(data)):
    row = ConvertListToSQLInsert(data.loc[i].tolist(), col_datatype)
    print(command.format(table="das", columns="dasd", values=','.join(row)))
    break

insert into das(dasd) values ('GSL-00022','CTL','GSLK2806N','GSLK2806N','GSL','GLS PHUONG NAM','1900-12-31 23:00:00','SALAN','Z','B','SG7984','K2806N','K2806N','TCC',null,'CTL','B08','1900-12-31 23:00:00','2019-06-29 07:59:00','2019-06-29 08:43:00','2019-06-29 08:42:00','1900-12-31 00:00:00','2019-06-29 08:00:00','D',null,0.0,'1900-12-31 23:00:00',0,null,0,null,'P',null,null,'1900-12-31 23:00:00',0.0,'1900-12-31 23:00:00','2019-06-28 16:00:00','2019-06-28 19:00:00')


In [76]:
def CheckDatatype(data, col_columns, col_datatype, change=True):
    for i in range(len(col_datatype)):
        if 'char' in col_datatype[i]:
#             print(col_columns[i], col_datatype[i], end='')
            try:
                data[col_columns[i]] = data[col_columns[i]].apply(str)
                str_type = col_datatype[i]
                numOfChar = str_type[str_type.index('(')+1 : str_type.index(')')]
                lstLen = [len(text) for text in data[col_columns[i]].unique().tolist()]
                if max(lstLen) > int(numOfChar):
                    col_datatype[i] = col_datatype[i].replace(numOfChar, str(max(lstLen)))
#                     raise ValueError(col_columns[i] + " has elements longer than " + col_datatype[i])
                else:
                    pass
#                     print('check')
            except:
                raise TypeError("Cant convert " +col_columns[i] + " to " + col_datatype[i])
    return data, col_columns, col_datatype

In [160]:
data, col_columns, col_datatype = CheckDatatype(data, col_columns, col_datatype)

ITEM_NO char(12)check
SITE_ID char(6)check
ARR_BY char(1)check
ARR_CAR char(12)check
FEL char(1)check
DEP_BY char(1)check
DEP_CAR char(19)check
ISO char(4)check
CATEGORY char(1)check
OWNER char(3)check
EXIT_VES_CD char(3)check
EXIT_VOYAGE char(12)check
PLACE_OF_DELIVERY char(8)check
PLACE_OF_RECEIPT char(8)check
ITEM_TYPE char(2)check
SLOT_CODE char(7)check
LINE_OPER char(7)check
AGENT char(3)check
DISCH_PORT char(5)check
FDISCH_PORT char(5)check
HIST_FLG char(3)check
LOAD_PORT char(5)check
LL_DISCH_PORT char(5)check


In [143]:
max([len(text) for text in data['DEP_CAR'].unique().tolist()])

19

In [156]:
col_columns.index('ISO')

10

In [117]:
text = 'char(40)'

In [119]:
text[text.index('(')+1 : text.index(')')]

'40'

In [35]:
i = -5
while i < 5:
    print(i/2)
except:
    print("Chia cho 0")

SyntaxError: invalid syntax (242151486.py, line 4)

In [176]:
data.loc[0]

VES_ID                         GSL-00022
SITE_ID                              CTL
TFC_CODE_E                     GSLK2806N
TFC_CODE_I                     GSLK2806N
VES_CD                               GSL
VES_NAME                  GLS PHUONG NAM
VES_DEP_DATE         1900-12-31 23:00:00
VES_SERVICE                        SALAN
VES_STATUS                             Z
VES_TYPE                               B
CALL_SIGN                         SG7984
IN_VOYAGE                         K2806N
OUT_VOYAGE                        K2806N
LINE_OPER                            TCC
AGENT                                nan
OWNER                                CTL
BERTH_NO                             B08
ACT_ANCHORAGE_TS     1900-12-31 23:00:00
ACT_BERTH_TS         2019-06-29 07:59:00
ACT_DEP_TS           2019-06-29 08:43:00
ACT_END_WORK_TS      2019-06-29 08:42:00
ACT_PILOT_TS         1900-12-31 00:00:00
ACT_START_WORK_TS    2019-06-29 08:00:00
OCEAN_INTERISLAND                      D
ALL_LINES       

In [186]:
data = data.fillna(value='null')

In [188]:
data.loc[9]

VES_ID                         T21-00807
SITE_ID                              CTL
TFC_CODE_E                   T21S260719X
TFC_CODE_I                       T21NONE
VES_CD                               T21
VES_NAME                     TAN CANG 21
VES_DEP_DATE         1900-12-31 23:00:00
VES_SERVICE                        SALAN
VES_STATUS                             Z
VES_TYPE                               B
CALL_SIGN                          QC-21
IN_VOYAGE                           NONE
OUT_VOYAGE                      S260719X
LINE_OPER                            TWT
AGENT                                nan
OWNER                                CTL
BERTH_NO                             B08
ACT_ANCHORAGE_TS     1900-12-31 23:00:00
ACT_BERTH_TS         2019-07-26 20:18:00
ACT_DEP_TS           2019-07-27 03:25:00
ACT_END_WORK_TS      2019-07-27 03:24:00
ACT_PILOT_TS         1900-12-31 00:00:00
ACT_START_WORK_TS    2019-07-26 20:19:00
OCEAN_INTERISLAND                      D
ALL_LINES       

In [344]:
connection.close()

In [197]:
result = cursor.execute("select * from VESSEL_DETAILS")

ProgrammingError: ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'VESSEL_DETAILS'. (208) (SQLExecDirectW)")

In [237]:
date = data.loc[0, 'CRT_TS']

In [239]:
data.loc[0]

OPERATION_METHOD           char(20)
CHARGE_CD                  char(12)
CATEGORY                   char(50)
SPEC_HDL_CD                 char(4)
OPERATION_NAME         varchar2(50)
CRT_TS                         date
SITE_ID                     char(6)
OPERATION_TYPE         varchar2(12)
TERMINAL_ID                char(12)
MOVE_REASON_CD              char(4)
DIRECT_IMP_EXP              char(1)
PLACE_OF_RECEIPT            char(3)
PLACE_OF_DELIVERY           char(3)
OUT_VOYAGE_INFO_FLG         char(1)
Name: 0, dtype: object

In [69]:
cursor.execute("insert into test(real_name) values ('%s')"%(CleanText(lst[0])))

In [27]:
name = "'a\r\nb'"
real_name = "'a\r\nb'"

In [29]:
"insert into test(real_name) values (%s)"%(name, real_name)

"insert into test values ('a\r\nb', 'a\r\nb')"

In [54]:
result = cursor.execute('select * from test')

ProgrammingError: The cursor's connection has been closed.

In [20]:
result.fetchall()

[('viet', 'First'),
 ('nemae', None),
 ('name', 'real_name'),
 ('test', 'dasd'),
 ('First line', 'Second\r\nline')]

In [135]:
connection.commit()
connection.close()

In [82]:
lst = data['COMMODITY'].tolist()

In [66]:
def CleanText(text):
    text = text.replace('\n', '\r\n')
    text = text.replace('_x000D_', '')
    text = text.replace('"', '')
    return text

In [68]:
'%s'%(CleanText(lst[0])

'SHIPPERS LOAD, COUNT & SEAL\r\nSAID TO CONTAIN\r\n1X20\r\n1,500 CARTONS\r\n\r\n15,0'

In [123]:
row[15] = "N' '"

In [113]:
row = ConvertListToSQLInsert(data.loc[0].tolist(), col_datatype)

In [125]:
cursor.execute('insert into DISCHARGE_LIST values (%s)'%(','.join(row)))

In [124]:
','.join(row)

"12767773,'BMOU1313845','NTV1812N','GP',null,null,6978595,null,'CTL','NTV-00040','NORTHERN VOLITION','KMT','KMT','KMT',N'SHIPPERS LOAD, COUNT & SEAL\r\nSAID TO CONTAIN\r\n1X20\r\n1,500 CARTONS\r\n15,0',N' ',null,null,'VNCTL'"